In [1]:
from helpers.models import Models
from helpers.llm_client import LLMClient
from helpers.parsers import license_parser
from helpers.functions import *
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score
# pd.set_option('display.max_colwidth', None)  # Show full column width

/home/jimbo/Desktop/GSoC24/repo/GSoC24/gsoc24env/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/jimbo/Desktop/GSoC24/repo/GSoC24/gsoc24env/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from collections import Counter
import string

text = '\n'.join(pd.read_csv('extras/license_information/license_dataset.csv')['License Text'])

standard_chars = set(string.ascii_letters + string.digits + string.punctuation)

non_standard_counts = Counter(char for char in text if char not in standard_chars)

sorted_counts = sorted(non_standard_counts.items(), key=lambda item: item[1], reverse=True)

for char, count in sorted_counts:
    print(f"'{char}': {count}")

In [ ]:
# notice file -> acknowledgement
# obligations

In [ ]:
create_license_dataset('extras/license_information/details')
client = LLMClient()
model = SentenceTransformer("all-mpnet-base-v2")
df = pd.read_csv('extras/dataset-labeled-1-75.csv')
df = extract_comments(df)
lines_tuple = get_top_similar_license_lines(\
        code_text = df.loc[50, 'file_comments'],
        licenses_file_path = 'extras/license_information/license_dataset.csv',
        model = model,
        top_k = 10,
        min_similarity = 60,
        double_semantic_search = True
    )
lines_tuple

## multi-qa-mpnet-base-dot-v1

In [ ]:
create_license_dataset('extras/license_information/details')
client = LLMClient()
model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")
df = pd.read_csv('extras/dataset-labeled-1-75.csv')
df = extract_comments(df)
for index, row in df.loc[:75].iterrows():
    lines_tuple = get_top_similar_license_lines(\
                    code_text = row['file_comments'],
                    licenses_file_path = 'extras/license_information/license_dataset.csv',
                    model = model,
                    top_k = 10,
                    double_semantic_search=True,
                    min_similarity=60
            )
    lines = [line[1] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'license_relevant_lines'] = lines
    lines = [line[2] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_names'] = lines
    lines = [line[3] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_ids'] = lines
for index, row in df.iterrows():
    df.loc[index, 'license_found'] = license_line_found(row['license_relevant_lines'], row['response'])
    df.loc[index, 'coverage'] = asses_coverage(row['license_relevant_lines'], row['response'])
print('accuracy:', accuracy_score([1] * 75, df.loc[0:74]['license_found']))
print('coverage: ', np.average(df.loc[0:74]['coverage']))

In [ ]:
miss_classifcations = df.loc[0:74][df['license_found'] == 0]
miss_classifcations[['file path', 'license_relevant_lines','response','predicted_license_names','scan results','predicted_license_ids']]

## all-mpnet-base-v2

In [3]:
create_license_dataset('extras/license_information/details')
client = LLMClient()
model = SentenceTransformer("all-mpnet-base-v2")
df = pd.read_csv('extras/dataset-labeled-1-75.csv')
df = extract_comments(df)
for index, row in df.loc[:75].iterrows():
    lines_tuple = get_top_similar_license_lines(\
                    code_text = row['file_comments'],
                    licenses_file_path = 'extras/license_information/license_dataset.csv',
                    model = model,
                    top_k = 10,
                    min_similarity = 60,
                    double_semantic_search = True
            )
    lines = [line[1] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'license_relevant_lines'] = lines
    lines = [line[2] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_names'] = lines
    lines = [line[3] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_ids'] = lines
for index, row in df[0:75].iterrows():
    df.loc[index, 'license_found'] = license_line_found(row['license_relevant_lines'], row['response'])
    df.loc[index, 'license_coverage'] = asses_coverage(row['license_relevant_lines'], row['response'])
    df.loc[index, 'predicted_license_accurate'] = predicted_license_found(row['predicted_license_ids'], row['scan results'])
    df.loc[index, 'predicted_licenses_covered'] = predicted_license_covered(row['predicted_license_ids'], row['scan results'])
print('License Line Found Accuracy:', accuracy_score([1] * 75, df.loc[0:74]['license_found']))
print('License Lines Found Coverage:', np.average(df.loc[0:74]['license_coverage']))
print('Predicted License Accuracy:', accuracy_score([1] * 75, df.loc[0:74]['predicted_license_accurate']))
print('Predicted Licenses Covered:', np.average(df.loc[0:74]['predicted_licenses_covered']))

License dataset file created successfully at extras/license_information/license_dataset.csv


/home/jimbo/Desktop/GSoC24/repo/GSoC24/gsoc24env/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


License Line Found Accuracy: 0.6
License Lines Found Coverage: 57.676190476190484
Predicted License Accuracy: 0.9333333333333333
Predicted Licenses Covered: 84.0


In [4]:
pd.set_option('display.max_colwidth', None)  # Show full column width
df[df['predicted_license_accurate'] == 0][['scan results', 'predicted_license_ids']]

,scan results,predicted_license_ids
11,GPL,Ferguson-Twofish
58,See-doc.OTHER,
59,See-doc.OTHER,ODbL-1.0
60,See-doc.OTHER,BSD-3-Clause-Modification
61,See-doc.OTHER,CECILL-C\nLucida-Bitmap-Fonts


In [ ]:
print(df.loc[48, 'file_comments'])

In [ ]:
create_license_dataset('extras/license_information/details')
client = LLMClient()
model = SentenceTransformer("all-mpnet-base-v2")
df = pd.read_csv('extras/dataset-labeled-1-75.csv')
df = extract_comments(df)
lines_tuple = get_top_similar_license_lines(\
                code_text = df.loc[48, 'file_comments'],
                licenses_file_path = 'extras/license_information/license_dataset.csv',
                model = model,
                top_k = 10,
                embedding_approach='license-embedding',
                double_semantic_search = True
    )
lines_tuple

In [ ]:
print(df.loc[63, 'file_comments'])

In [ ]:
miss_classifcations = df.loc[0:74][df['coverage'] != 100]
miss_classifcations[['file path', 'license_relevant_lines','response','predicted_license_names','scan results','predicted_license_ids']]

In [ ]:
df.loc[0:74][df['license_found'] == 1][['file path', 'license_relevant_lines','response','predicted_license_names','scan results','predicted_license_ids']]

In [ ]:
licenses = pd.read_csv('extras/license_information/license_dataset.csv')

## paraphrase-MiniLM-L3-v2

In [ ]:
create_license_dataset('extras/license_information/details')
client = LLMClient()
model = SentenceTransformer("paraphrase-MiniLM-L3-v2")
df = pd.read_csv('extras/dataset-labeled-1-75.csv')
for index, row in df.loc[:75].iterrows():
    lines_tuple = get_top_similar_license_lines(\
                    code_text = row['file_comments'],
                    licenses_file_path = 'extras/license_information/license_dataset.csv',
                    model = model,
                    top_k = 10,
                    embedding_approach='license-embedding'
            )
    lines = [line[2] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'license_relevant_lines'] = lines
    lines = [line[3] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_names'] = lines
    lines = [line[4] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_ids'] = lines
for index, row in df.iterrows():
    df.loc[index, 'license_found'] = license_line_found(row['license_relevant_lines'], row['response'])
    df.loc[index, 'coverage'] = asses_coverage(row['license_relevant_lines'], row['response'])
print('accuracy:', accuracy_score([1] * 75, df.loc[0:74]['license_found']))
print('coverage: ', np.average(df.loc[0:74]['coverage']))

In [ ]:
miss_classifcations = df.loc[0:74][df['license_found'] == 0]
miss_classifcations[['file path', 'license_relevant_lines','response','predicted_license_names','scan results','predicted_license_ids']]

## all-MiniLM-L12-v2 

In [ ]:
create_license_dataset('extras/license_information/details')
client = LLMClient()
model = SentenceTransformer("all-MiniLM-L12-v2")
df = pd.read_csv('extras/dataset-labeled-1-75.csv')
for index, row in df.loc[:75].iterrows():
    lines_tuple = get_top_similar_license_lines(\
                    code_text = row['file_comments'],
                    licenses_file_path = 'extras/license_information/license_dataset.csv',
                    model = model,
                    top_k = 10,
                    embedding_approach='license-embedding'
            )
    lines = [line[2] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'license_relevant_lines'] = lines
    lines = [line[3] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_names'] = lines
    lines = [line[4] for line in lines_tuple]
    lines = '\n'.join(lines)
    df.loc[index, 'predicted_license_ids'] = lines
for index, row in df.iterrows():
    df.loc[index, 'license_found'] = license_line_found(row['license_relevant_lines'], row['response'])
    df.loc[index, 'coverage'] = asses_coverage(row['license_relevant_lines'], row['response'])
print('accuracy:', accuracy_score([1] * 75, df.loc[0:74]['license_found']))
print('coverage: ', np.average(df.loc[0:74]['coverage']))

In [ ]:
miss_classifcations = df.loc[0:74][df['license_found'] == 0]
miss_classifcations[['file path', 'license_relevant_lines','response','predicted_license_names','scan results','predicted_license_ids']]

In [ ]:
idx = 1
print(df[df['license_found'] == 1][['file path', 'license_relevant_lines','response','predicted_license_names','scan results','predicted_license_ids']].loc[idx])
print('----')
print(df.loc[idx, 'predicted_license_ids'])